## Homework
In the workshop, we extracted contents from two pages in notion titled "Workshop: Benefits and Perks" and "Workshop: Working hours, PTO, and Vacation".

Repeat the same process for a third page titled "Homework: Employee handbook" (hidden from public view, but accessible via API key):
- Modify the REST API source to extract only this page.
- Write the output into a separate table called "homework".
- Remember to update the table name in all cells where you connect to a lancedb table.
To do this you can use the workshop Colab as a basis.


In [3]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

In [98]:
!yes | dlt init rest_api lancedb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
No files to update, exiting


Import environment variables

In [99]:
import os
from google.colab import userdata

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = userdata.get("SOURCES__REST_API__NOTION__API_KEY")

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

Import necessary modules

In [100]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

Configure the dlt notion rest api source - ```RESTAPIConfig```

In [101]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    raw_api_key = dlt.secrets["sources.rest_api.notion.api_key"]
    clean_api_key = raw_api_key.strip()  # Удаляем лишние символы

    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": clean_api_key  # Используем очищенный токен
            },
            "headers": {
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "homework",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config, name="employee_handbook")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="homework",
    write_disposition="replace",
    primary_key="block_id",
    columns={"last_edited_time": {"dedup_sort": "desc"}}
)
def rest_api_notion_incremental(
    last_edited_time=dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z", primary_key=("block_id"))
):
    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not len(block["content"]):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages___homework",
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="replace",
    )
    print(load_info)

load_notion()

Pipeline company_policies load step completed in 2.80 seconds
1 load package(s) were loaded to destination LanceDB and into dataset notion_pages_homework
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7e9c350045b0> location to store data
Load package 1721592615.9826424 is LOADED and contains no failed jobs


Let's see an output

In [102]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages_homework___homework")

homework_df = dbtable.to_pandas()

In [103]:
dbtable

LanceTable(connection=LanceDBConnection(/content/.lancedb), name="notion_pages_homework___homework")

In [104]:
print(db.table_names())

['notion_pages____dlt_loads', 'notion_pages____dlt_pipeline_state', 'notion_pages____dlt_version', 'notion_pages___dltSentinelTable', 'notion_pages___employee_handbook', 'notion_pages___homework', 'notion_pages_homework____dlt_loads', 'notion_pages_homework____dlt_pipeline_state', 'notion_pages_homework____dlt_version', 'notion_pages_homework___dltSentinelTable', 'notion_pages_homework___homework']


In [105]:
homework_df.shape

(17, 9)

## Q1. Rows in LanceDB
How many rows does the lancedb table "notion_pages__homework" have?

- 14
- 15
- 16
- 17

### Answer:
- 17

## Q2. Running the Pipeline: Last edited time
In the demo, we created an incremental dlt resource rest_api_notion_incremental that keeps track of last_edited_time. What value does it store after you've run your pipeline once? (Hint: you will be able to get this value by performing some aggregation function on the column last_edited_time of the table)

- Timestamp('2024-07-05 22:34:00+0000', tz='UTC') (OR "2024-07-05T22:34:00.000Z")
- Timestamp('2024-07-05 23:33:00+0000', tz='UTC') (OR "2024-07-05T23:33:00.000Z")
- Timestamp('2024-07-05 23:52:00+0000', tz='UTC') (OR "2024-07-05T23:52:00.000Z")
- Timestamp('2024-07-05 22:56:00+0000', tz='UTC') (OR "2024-07-05T22:56:00.000Z")

database with vector representation

In [106]:
homework_df

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,a7932901-83cf-5ef3-a791-753b41a13299,"[-0.024265623, 0.04746083, -0.011796438, 0.063...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-21 20:10:20.460545+00:00,1721592615.9826424,7Xt5lLJo19qXjQ
1,de531c5c-7e82-5f5c-b614-f9d4ecb0000e,"[-0.049661573, 0.10853516, -0.0097625945, -0.0...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-21 20:10:20.467598+00:00,1721592615.9826424,6mFenikEyzgSZw
2,8be51d4e-a91b-5fe2-8d8c-fe9afea4d0e9,"[-0.06316319, 0.17331506, 0.025351718, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-21 20:10:20.468093+00:00,1721592615.9826424,T8vTQgGT283LGw
3,44c2f6fe-5a08-5ce9-a1af-81099e32d11c,"[-0.10974315, 0.10586075, 0.0032906067, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-21 20:10:20.468428+00:00,1721592615.9826424,d4CQpj3rKZj1qA
4,74f486d2-f271-58f8-a347-3aa45aa09f3e,"[0.052423306, -0.06457594, 0.065863, 0.0145438...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-21 20:10:20.468772+00:00,1721592615.9826424,vAPAA5Sz0XkdZA
5,f2aa1fd8-037f-5430-b41d-65863cb99546,"[0.0005233448, -0.054883398, 0.043573365, -0.0...",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-21 20:10:20.469159+00:00,1721592615.9826424,6x6vJO8fnqfD2g
6,007cbe0f-7ff7-5a08-8f7e-3d20b97f407b,"[0.03802632, -0.021509668, 0.0475278, 0.064706...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-21 20:10:20.469508+00:00,1721592615.9826424,Jh3mVsLgrahsIw
7,3112ea01-282a-5b1d-8768-17db39db5bb2,"[-0.058588073, -0.07540443, 0.033775173, 0.009...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-21 20:10:20.469876+00:00,1721592615.9826424,SawA8uJbVzu34w
8,bda7b0e0-a9c5-5928-b5e2-eee965dad000,"[-0.01359926, 0.004753031, 0.024835143, 0.0159...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-21 20:10:20.470252+00:00,1721592615.9826424,yY6UGJpYgdiT0g
9,3aee26eb-c316-5799-bcde-c573882fbb6b,"[0.032060914, 0.02424462, 0.008471355, 0.03179...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-21 20:10:20.470579+00:00,1721592615.9826424,D0yZDOCXnMhHag


In [107]:
homework_df['last_edited_time'].max()

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

### Answer:
- Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

## Q3. Ask the Assistant
Find out with the help of the AI assistant: how many PTO days are the employees entitled to in a year?

- 20
- 25
- 30
- 35

Install Ollama into the notebook's local runtime

In [23]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Start Ollama using ollama serve. This needs to run in the backgound - so we run it using nohup

In [24]:
!nohup ollama serve > nohup.out 2>&1 &

Pull the desired model. We're going to be using llama1-uncensored

In [25]:
%%capture
!ollama pull llama2-uncensored

pip install ollama and import it

In [26]:
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [93]:
import ollama

Write a function that can retrieve content from lancedb relevant to the user query

In [108]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

Define a very basic RAG. We define a simple system prompt, retrieve the relevant context for the user query with the function defined above and then send the user question and the context to the llama2-uncensored model.

In [109]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages_homework___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

 RUN the RAG!

In [110]:
main()

You:  how many PTO days are the employees entitled to in a year?


ConnectError: [Errno 111] Connection refused